# 优化超参数

有许多机器学习算法需要超*参数*（影响训练但无法通过训练数据本身确定的参数值）。例如，训练逻辑回归模型时，可以使用正则化率超参数来抵消模型中的偏差；或者训练卷积神经网络时，可以使用学习速率和批大小等超参数来分别控制权重的调整方式以及以小批量方式处理的数据项数量。超参数值的选择会显著影响已训练模型的性能或训练该模型所需的时间；通常需要尝试多种组合以找到最佳解决方案。

在本例中，你将使用带有单个超参数的简单逻辑回归模型示例，但是原则适用于可使用 Azure 机器学习训练的任何类型的模型。

## 连接到工作区

你首先需要使用 Azure ML SDK 连接到工作区。

> **注意**：如果 Azure 订阅的身份验证会话在你完成上一练习后已过期，系统将提示你重新进行身份验证。

In [ ]:
import azureml.core
from azureml.core import Workspace

# 从保存的配置文件加载工作区
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## 准备试验数据

在本实验中，你将使用包含糖尿病患者详细信息的数据集。运行以下单元格以创建此数据集（如果在上一个实验中创建了此数据集，则代码将创建一个新版本）

In [ ]:
from azureml.core import Dataset

default_ds = ws.get_default_datastore()

if 'diabetes dataset' not in ws.datasets:
    default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # 将糖尿病 csv 文件上传到 /data 中
                        target_path='diabetes-data/', # 将其放在数据存储的文件夹路径中
                        overwrite=True, # 替换名称相同的现有文件
                        show_progress=True)

    #从数据存储上的路径创建表格数据集（这可能需要一些时间）
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

    # 注册表格数据集
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='diabetes dataset',
                                description='diabetes data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')

## 准备训练脚本

首先，为将用于训练逻辑回归模型的训练脚本创建一个文件夹。

In [ ]:
import os

experiment_folder = 'diabetes_training-hyperdrive'
os.makedirs(experiment_folder, exist_ok=True)

print('Folder ready.')

现在创建 Python 脚本以训练模型。此脚本必须包括：

- 每个要优化的超参数（在本例中只有正则化超参数）的参数
- 用于记录要优化的性能指标的代码（在本例中将记录 AUC 和准确度，以便可以选择针对其中一项优化模型）

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# 导入库
import argparse
import joblib
import os
from azureml.core import Run
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# 设置正则化参数
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01, help='regularization rate')
args = parser.parse_args()
reg = args.reg_rate

# 获取试验运行上下文
run = Run.get_context()

# 加载糖尿病数据集
print("Loading Data...")
diabetes = run.input_datasets['diabetes'].to_pandas_dataframe() # 从估算器输入中获取训练数据

# 分隔特征和标签
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# 将数据拆分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# 训练逻辑回归模型
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# 计算精度
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# 计算 AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

os.makedirs('outputs', exist_ok=True)
# 注意，保存在输出文件夹中的文件会自动上传到试验记录
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

## 准备计算目标

云计算的好处之一是它可以按需扩展，使你能够预配足够的计算资源来并行处理试验的多次运行，每次运行都具有不同的超参数值。

你将使用在前面的实验中创建的 **aml-cluster Azure** 机器学习计算群集（如果不存在，则将创建它）。

> **重要事项**：在运行以下代码之前，请先在代码中将*“你的计算群集”*更改为你的计算群集的名称！群集名称必须是长度在 2 到 16 个字符之间的全局唯一名称。有效字符是字母、数字和 - 字符。

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "your-compute-cluster"

try:
    # 检查现有的计算目标
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # 如果尚不存在，请创建它
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)
    

## 运行 *Hyperdrive 试验*

Azure 机器学习包括通过 *Hyperdrive* 试验提供的超参数优化功能。这些试验启动了多个子运行，每个子运行都有不同的超参数组合。可以确定生成最佳模型的运行（由要优化的记录目标性能指标确定），并选择其经过训练的模型进行注册和部署。

In [ ]:
from azureml.core import Experiment
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive import GridParameterSampling, BanditPolicy, HyperDriveConfig, PrimaryMetricGoal, choice
from azureml.widgets import RunDetails


# 对一系列参数值进行采样
params = GridParameterSampling(
    {
        # 只有一个参数，因此网格采样将尝试每个值 - 通过多个参数，它将尝试每种组合
        '--regularization': choice(0.001, 0.005, 0.01, 0.05, 0.1, 1.0)
    }
)


# 获取训练数据集
diabetes_ds = ws.datasets.get("diabetes dataset")

# 创建使用远程计算的估算器
hyper_estimator = SKLearn(source_directory=experiment_folder,
                          inputs=[diabetes_ds.as_named_input('diabetes')], # 将数据集作为输入传递…
                          pip_packages=['azureml-sdk'], # ...因此我们需要 azureml-dataprep（它在 SDK 中！）
                          entry_script='diabetes_training.py',
                          compute_target = training_cluster,)

# 配置 hyperdrive 设置
hyperdrive = HyperDriveConfig(estimator=hyper_estimator, 
                          hyperparameter_sampling=params, 
                          policy=None, 
                          primary_metric_name='AUC', 
                          primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                          max_total_runs=6,
                          max_concurrent_runs=4)

# 运行试验
experiment = Experiment(workspace = ws, name = 'diabates_training_hyperdrive')
run = experiment.submit(config=hyperdrive)

# 显示试验运行时笔记本中的状态
RunDetails(run).show()
run.wait_for_completion()

可在上面的小组件中查看试验运行状态。还可以在 [Azure 机器学习工作室](https://ml.azure.com)中查看主要的 Hyperdrive 试验运行及其子运行。

> **注意**：小组件可能无法刷新。运行完成后，你将在小组件下方看到摘要信息。

## 确定最佳性能运行

完成所有运行后，可以根据指定的性能指标找到最佳运行（在本例中为具有最佳 AUC 的运行）。

In [ ]:
for child_run in run.get_children_sorted_by_primary_metric():
    print(child_run)

best_run = run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details() ['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print(' -AUC:', best_run_metrics['AUC'])
print(' -Accuracy:', best_run_metrics['Accuracy'])
print(' -Regularization Rate:',parameter_values)

你已找到最佳运行，现在可以注册它所训练的模型。

In [ ]:
from azureml.core import Model

# 注册模型
best_run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                        tags={'Training context':'Hyperdrive'},
                        properties={'AUC': best_run_metrics['AUC'], 'Accuracy': best_run_metrics['Accuracy']})

# 列出已注册的模型
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

> **更多信息**：有关 Hyperdrive 的详细信息，请参阅 [Azure ML 文档](https://docs.microsoft.com/azure/machine-learning/how-to-tune-hyperparameters)。